### COMP 6315 

### Feature Selection en Clasificacion Supervisada
#### Edgar Acuna

Uso de feature_selection de scikit-learn, del modulo feature selection de Arizona State University, del modulo skrebate para feature selection y del modulo Orange.

### I-Usando la prueba de Chi-Square para ver si hay independencia entre la feature y la clase.

De preferencia los datos deben estar discretizados y ademas con dos clases es mejor

Personalmente no la recomiendo porque la priueba de Chisquare es aproximada y no muy robusta

In [1]:
#Chi-Square usando el  modulo de Feature selction de Scikit-learn
import pandas as pd
import numpy as np
import math as m
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, SelectPercentile, f_classif, mutual_info_classif
# load data
url= "http://academic.uprm.edu/eacuna/diabetes.dat"
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
data = pd.read_table(url, names=names,header=None)
y=data['class']
X=data.iloc[:,0:8]

In [2]:
# Funcion auxiliar para discretizar cualquier columna de un dataframe
def disc_col_ew(df,str,k,out):
    df1=df[str]
    bins=np.linspace(df1.min(), df1.max(),k)
    if out=="num":
        df1=pd.cut(df1,bins=bins,include_lowest=True, right=True,labels=False)
    else:
        bins[0]=float('-inf')
        bins[k-1]=float('inf')
        df1=pd.cut(df1,bins=bins,include_lowest=True, right=True)  
    return df1
# funcion auxiliar para determinar el numero optimo de intervalos segun la formula de scott
def nclass_scott(x):
    h=3.5*(np.var(x,ddof=1)**.5)*len(x)**(-.3333)
    intervals=m.ceil((max(x)-min(x))/h)
    return int(intervals)

In [3]:
#Funcion para discretizar todas las colunmnas de un dataframe
def disc_ew(df,out):
    name=df.columns.tolist()
    disc=pd.DataFrame()
    for name in df.columns.tolist():
        k=nclass_scott(df[name])
        disc[name]=disc_col_ew(df,name,k,out)
    return disc

In [4]:
#discretizando las columnas de la matriz predictora X de diabetes
diab_disc=disc_ew(X,out="num")

In [5]:
y1=y.as_matrix()
X1=diab_disc.as_matrix()
# feature extraction
test = SelectKBest(score_func=chi2, k=3)
fit = test.fit(X1, y1)
# summarize scores
np.set_printoptions(precision=3)
print(fit.scores_)
features= fit.transform(X1)
# Imprime los datos de las tres mejores features
print(features)

[ 105.387  120.206    2.87    10.086   59.232   42.746   62.639  115.924]
[[ 4 11  6]
 [ 0  6  2]
 [ 6 14  2]
 ..., 
 [ 3  9  1]
 [ 0 10  5]
 [ 0  7  0]]


Comentario: Las tres mejores variables con la prueba de Chi-square son plas(2), age(8) y preg(1) por tener el Chi-Square mas alto.

II-Usando la prueba de estadistica de F para comparar dos o mas grupos

Se puede usar la libreria sciki-learn o la libreria de la ASU


### II-Usando la prueba de estadistica de F para comparar dos o mas grupos

#### Se puede usar la libreria sciki-learn o la libreria de la ASU
#### Seleccion  de Features usando los p-values de la F-test como score
#### Aqui usamos scikit-learn y se selecciona el 30% de deatures con el mas alto score

In [6]:
selector = SelectPercentile(f_classif, percentile=30)
fit=selector.fit(X, y)
scores = -np.log10(selector.pvalues_)
scores /= scores.max()
print(scores)
features= fit.transform(X)
# Imprime los datos de las tres mejores features
print(features)

[ 0.221  1.     0.027  0.034  0.084  0.378  0.14   0.253]
[[ 148.    33.6   50. ]
 [  85.    26.6   31. ]
 [ 183.    23.3   32. ]
 ..., 
 [ 121.    26.2   30. ]
 [ 126.    30.1   47. ]
 [  93.    30.4   23. ]]


Comentario: Las tres, mejores variables con la prueba de F son plas,mass y age

In [7]:
#Usando la libreria Feature selection de la ASU
features, labels = X.values, data['class'].values
X_train, X_test, y_train, y_test = train_test_split(features, labels)
from skfeature.function.statistical_based import f_score
scoref = f_score.f_score(X_train, y_train)
idx = f_score.feature_ranking(scoref)
print(scoref,idx)

(array([  25.624,  178.853,    1.976,    2.186,    6.758,   68.07 ,
         15.168,   30.702]), array([1, 5, 7, 0, 6, 4, 3, 2], dtype=int64))


Comentario: las tres variables mas importantes con la prueba de F son: plas(2),mass(6) y age (8)

### III- Usando Mutual Information (Entropia)

Libreria scikit-learn


In [8]:
# Univariate feature selection with Mutual Information
scores = mutual_info_classif(X,y)
scores /= scores.max()
print(scores)

[ 0.21   1.     0.     0.164  0.386  0.643  0.135  0.382]



Comentario: Las tres variables usando el criterio de Mutual Information son: plas(2), mass(6) y age(8)

### IV-Usando ReliefF de la libreria skrebate
 

In [9]:
from skrebate import ReliefF
from sklearn.model_selection import train_test_split

df=data.drop('class',axis=1)
#Normalizando las predictoras
df_norm=(df - df.min()) / (df.max() - df.min())
features, labels = df_norm.values, data['class'].values

In [10]:
X_train, X_test, y_train, y_test = train_test_split(features, labels)

fs = ReliefF(n_neighbors=10)
fs.fit(X_train, y_train)

for feature_name, feature_score in zip(df.columns,fs.feature_importances_):
    print (feature_name, feature_score)

('preg', 38.928571428571438)
('plas', 77.454773869346795)
('pres', 12.348360655737721)
('skin', 55.033333333333324)
('test', 18.517473118279579)
('mass', 47.304769001490286)
('pedi', 19.15992892047981)
('age', 48.205882352941138)


Comentario: las tres variables mas importantes con el ReliefF son: plas,mass y skin

In [11]:
#Usando la libreria feature selection de la ASU
from skfeature.function.similarity_based import reliefF
score_relief=reliefF.reliefF(X_train,y_train)
feat=reliefF.feature_ranking(score_relief)
print(score_relief,feat)

(array([  4.786,  12.749,   4.308,  11.993,   1.77 ,  10.177,   2.811,
         7.894]), array([1, 3, 5, 7, 0, 2, 6, 4], dtype=int64))


 Comentario: las tres variables mas importantes con el RelieF son: plas, mass y skin

In [12]:
#Usando el Relief de Orange
import Orange
df = Orange.data.Table("diabetes")

def print_best_3(ma):
    for m in ma[:3]:
        print "%5.3f %s" % (m[1], m[0])

print 'Feature scores for best three features (scored individually):'
#ReliefF usando 10 vecinos mas cercanos y  una muestrd m-100 para updating de los pesos
meas = Orange.feature.scoring.Relief(k=10, m=100)
mr = [ (a.name, meas(a, df)) for a in df.domain.attributes]
mr.sort(key=lambda x: -x[1]) #sort decreasingly by the score
print_best_3(mr)


Feature scores for best three features (scored individually):
0.036 plas
0.026 age
0.015 mass


In [13]:
# Usando el reliefF de la ASU
#load data
url= "http://academic.uprm.edu/eacuna/bupa.dat"
names = ['mcv', 'alkphos', 'sgpt', 'aspar', 'gammagt', 'drinks', 'class']
data = pd.read_table(url, names=names,header=None)
y=data['class']
X=data.iloc[:,0:6]
y1=y.as_matrix()
X1=X.as_matrix()
features, labels = X.values, data['class'].values

In [14]:
X_train, X_test, y_train, y_test = train_test_split(features, labels)
score_relief=reliefF.reliefF(X_train,y_train)
feat=reliefF.feature_ranking(score_relief)
print(score_relief,feat)

(array([   9.8,   11.8,  258.8,  100.6,  -26. ,   51.6]), array([2, 3, 5, 1, 0, 4], dtype=int64))


El Rielief de ASU recomienda sgpt,aspar y drinks como las mejoras predictoras(***)

In [15]:
fs = ReliefF(n_neighbors=10)
fs.fit(X_train, y_train)

for feature_name, feature_score in zip(data.columns,fs.feature_importances_):
    print (feature_name, feature_score)

('mcv', 6.8999999999999924)
('alkphos', -2.6608695652173888)
('sgpt', 5.5794701986754962)
('aspar', 1.8972602739726028)
('gammagt', 6.7705479452054806)
('drinks', 6.1250000000000053)


El relielf de skrebate recomienda gammagt, sgpt y drinks

In [16]:
url='http://academic.uprm.edu/eacuna/landsat.txt'
data = pd.read_table(url, header=None,delim_whitespace=True)
y=data.iloc[:,36]
X=data.iloc[:,0:36]
y1=y.as_matrix()
X1=X.as_matrix()
features, labels = X.values, y.values
X_train, X_test, y_train, y_test = train_test_split(features, labels)

In [17]:
#usando Relief de la ASU
score_relief=reliefF.reliefF(X_train,y_train)
feat=reliefF.feature_ranking(score_relief)
print(score_relief,feat)

(array([  941029.049,  1235293.399,   922072.516,  1113361.246,
         892324.03 ,  1191842.818,   853138.339,  1004128.13 ,
         995782.317,  1357151.35 ,   861505.329,  1171701.683,
         973918.465,  1239353.523,   886857.916,  1069429.199,
         964050.236,  1292473.082,   843190.446,  1000798.569,
        1033231.163,  1445319.275,   794793.958,  1072131.319,
         975229.727,  1280813.083,   861520.985,  1039042.356,
         926222.435,  1276662.863,   792328.425,   973153.352,
         983277.312,  1395561.086,   850211.81 ,  1102658.697]), array([21, 33,  9, 17, 25, 29, 13,  1,  5, 11,  3, 35, 23, 15, 27, 20,  7,
       19,  8, 32, 24, 12, 31, 16,  0, 28,  2,  4, 14, 26, 10,  6, 34, 18,
       22, 30], dtype=int64))


In [18]:
#Usando el Relief de Orange
import Orange
df = Orange.data.Table("landsat")

def print_best_3(ma):
    for m in ma[:10]:
        print "%5.3f %s" % (m[1], m[0])

print 'Feature scores for best three features (scored individually):'
#ReliefF usando 10 vecinos mas cercanos y  una muestrd m-100 para updating de los pesos
meas = Orange.feature.scoring.Relief(k=1, m=100)
mr = [ (a.name, meas(a, df)) for a in df.domain.attributes]
mr.sort(key=lambda x: -x[1]) #sort decreasingly by the score
print_best_3(mr)


Feature scores for best three features (scored individually):
0.113 a17
0.109 a13
0.109 a21
0.108 a1
0.106 a33
0.102 a9
0.100 a25
0.097 a29
0.095 a5
0.095 a14
